In [ ]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

<b>Load and check your API keys</b>
</br>
<b>- - - - - - - - - - - - - - - -</b>

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

# Function to check and display API key status
def check_api_key(key_name):
    key = os.getenv(key_name)
    
    if key:
        # Always show the first 7 characters of the key
        print(f"✓ {key_name} API Key exists and begins... ({key[:7]})")
        return True
    else:
        print(f"⚠️ {key_name} API Key not set")
        return False

# Check each API key (the function now returns True or False)
has_openai = check_api_key('OPENAI_API_KEY')
has_anthropic = check_api_key('ANTHROPIC_API_KEY')
has_google = check_api_key('GOOGLE_API_KEY')
has_deepseek = check_api_key('DEEPSEEK_API_KEY')
has_groq = check_api_key('GROQ_API_KEY')

<b>Input for travel planner</b></br>
Describe yourself, your travel companions, and the destination you plan to visit.
</br>
<b>- - - - - - - - - - - - - - - -</b>

In [4]:
# Provide a description of you or your family. Age, interests, etc.
person_description = "family with a 3 year-old"
# Provide the name of the specific destination or attraction and country
destination = "Belgium, Brussels"

<b>- - - - - - - - - - - - - - - -</b>

In [5]:
prompt = f"""
Given the following description of a person or family:
{person_description}

And the requested travel destination or attraction:
{destination}

Provide a concise response including:

1. Fit rating (1-10) specifically for this person or family.
2. One compelling positive reason why this destination suits them.
3. One notable drawback they should consider before visiting.
4. One important additional aspect to consider related to this location.
5. Suggest a few additional places that might also be of interest to them that are very close to the destination.
"""

In [ ]:
def run_prompt_on_available_models(prompt):
    """
    Run a prompt on all available AI models based on API keys.
    Continues processing even if some models fail.
    """
    results = {}
    api_response = [{"role": "user", "content": prompt}]
    
    # OpenAI
    if check_api_key('OPENAI_API_KEY'):
        try:
            model_name = "gpt-4o-mini"
            openai_client = OpenAI()
            response = openai_client.chat.completions.create(model=model_name, messages=api_response)
            results[model_name] = response.choices[0].message.content
            print(f"✓ Got response from {model_name}")
        except Exception as e:
            print(f"⚠️ Error with {model_name}: {str(e)}")
            # Continue with other models
    
    # Anthropic
    if check_api_key('ANTHROPIC_API_KEY'):
        try:
            model_name = "claude-3-7-sonnet-latest"
            # Create new client each time
            claude = Anthropic()
            
            # Use messages directly 
            response = claude.messages.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1000
            )
            results[model_name] = response.content[0].text
            print(f"✓ Got response from {model_name}")
        except Exception as e:
            print(f"⚠️ Error with {model_name}: {str(e)}")
            # Continue with other models
    
    # Google
    if check_api_key('GOOGLE_API_KEY'):
        try:
            model_name = "gemini-2.0-flash"
            google_api_key = os.getenv('GOOGLE_API_KEY')
            gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
            response = gemini.chat.completions.create(model=model_name, messages=api_response)
            results[model_name] = response.choices[0].message.content
            print(f"✓ Got response from {model_name}")
        except Exception as e:
            print(f"⚠️ Error with {model_name}: {str(e)}")
            # Continue with other models
    
    # DeepSeek
    if check_api_key('DEEPSEEK_API_KEY'):
        try:
            model_name = "deepseek-chat"
            deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
            deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
            response = deepseek.chat.completions.create(model=model_name, messages=api_response)
            results[model_name] = response.choices[0].message.content
            print(f"✓ Got response from {model_name}")
        except Exception as e:
            print(f"⚠️ Error with {model_name}: {str(e)}")
            # Continue with other models
    
    # Groq
    if check_api_key('GROQ_API_KEY'):
        try:
            model_name = "llama-3.3-70b-versatile"
            groq_api_key = os.getenv('GROQ_API_KEY')
            groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
            response = groq.chat.completions.create(model=model_name, messages=api_response)
            results[model_name] = response.choices[0].message.content
            print(f"✓ Got response from {model_name}")
        except Exception as e:
            print(f"⚠️ Error with {model_name}: {str(e)}")
            # Continue with other models
    
    # Check if we got any responses
    if not results:
        print("⚠️ No models were able to provide a response")
    
    return results

# Get responses from all available models
model_responses = run_prompt_on_available_models(prompt)

# Display the results
for model, answer in model_responses.items():
    display(Markdown(f"## Response from {model}\n\n{answer}"))

<b>Sythesize answers from all models into one</b>
</br>
<b>- - - - - - - - - - - - - - - -</b>

In [ ]:
# Create a synthesis prompt
synthesis_prompt = f"""
Here are the responses from different models:
"""

# Add each model's response to the synthesis prompt without mentioning model names
for index, (model, response) in enumerate(model_responses.items()):
    synthesis_prompt += f"\n--- Response {index+1} ---\n{response}\n"

synthesis_prompt += """
Please synthesize these responses into one comprehensive answer that:
1. Captures the best insights from each response
2. Resolves any contradictions between responses
3. Presents a clear and coherent final answer
4. Maintains the same format as the original responses (numbered list format)
5.Compiles all additional places mentioned by all models 

Your synthesized response:
"""

# Create the synthesis
if check_api_key('OPENAI_API_KEY'):
    try:
        openai_client = OpenAI()
        synthesis_response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": synthesis_prompt}]
        )
        synthesized_answer = synthesis_response.choices[0].message.content
        print("✓ Successfully synthesized responses with gpt-4o-mini")
        
        # Display the synthesized answer
        display(Markdown("## Synthesized Answer\n\n" + synthesized_answer))
    except Exception as e:
        print(f"⚠️ Error synthesizing responses with gpt-4o-mini: {str(e)}")
else:
    print("⚠️ OpenAI API key not available, cannot synthesize responses")